In [23]:
import numpy as np
from matplotlib import pyplot as plt

import pickle as pkl
from matplotlib import style
import networkx as nx

from jax import jit, jacfwd
from FormationControl import formation_control_force
from GradientEstimation import total_grad_est
from utils.MutualSepPathPlanning import mutual_separation_path_planning,sep_func
style.use('fivethirtyeight')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
def circulant(i,q,p,prev,post,undirected=False):
    """
        Generate a circulant graph with len(p) nodes, node i connected with [i-prev:i+post],i-prev and i+post included but self-loop eliminated.
    """
    n = len(p)
    G = nx.DiGraph()
    edges = [(j%n,i) for i in range(n) for j in range(i-prev,i+post+1)]
    G.add_edges_from(edges)
    G.remove_edges_from(nx.selfloop_edges(G))
    if undirected:
        G = G.to_undirected()
    return G
def fc_comm_network(i,qs,ps):
    """
        Given the iteration number i, source location qs, sensor location ps,
        return the current communication network G.
        
        This variation returns a fully connected network with number of nodes=len(ps),
        without self-loops.
    """
    A = np.ones([len(ps),len(ps)])-np.eye(len(ps))
    return nx.from_numpy_matrix(A,create_using=nx.DiGraph)

In [142]:
class CircularFormationController:
    
    def __init__(self):
       
        # Plan
        R = 0.8 # The radius of the circular formation
        planning_timesteps = 1
        max_linear_speed = 0.1
        planning_dt = 1
        xlim = (-np.inf,np.inf)
        ylim = xlim
        
        f_dLdp = jit(jacfwd(sep_func))
        self.MP = lambda ps,y:mutual_separation_path_planning(\
                                                R,ps,\
                                                planning_timesteps,\
                                                max_linear_speed,\
                                                planning_dt,\
                                                y,\
                                                xlim = xlim,\
                                                ylim = ylim,\
                                                 f_dLdp=f_dLdp)
    
        
    def update(self,my_loc,my_meas,neighbor_loc,neighbor_meas,):
        
        ps = np.vstack([my_loc,neighbor_loc])
        y = np.hstack([my_meas,neighbor_meas])
        waypoints = self.MP(ps,y) 
            
        return waypoints[0][0]
        

In [143]:
class VirtualSensor:
    def __init__(self,C1,C0,k,b,noise_std = 0.01):
        self._C1 = C1
        self._C0 = C0
        self._k = k
        self._b = b
        self._sig = noise_std
        self._min_read = 1e-5

        
    def measurement(self,q,p):
        dists=np.linalg.norm(q-p,axis=1)
        
        y = self._k*(dists-self._C1)**self._b\
        +self._C0 + np.random.randn(dists.shape[0])*self._sig
        
        return np.max([self._min_read*np.ones(len(y)),y],axis=0)

In [150]:
def get_adj_list(q,ps,G):
    a_list = []
    for i in G.nodes():
        a_list.append([k for k in G[i]])
    a_list = np.array(a_list)
    
    return a_list

C1=-0.3 # Setting C1 as a negative number mitigates the blowing-up effect when the sensors are close to the source.
C0=0
k=1
b=-2
vs = VirtualSensor(C1,C0,k,b)

N_iter = 100
N_trials = 100


N_sen = 10

controller = CircularFormationController()
comm_network_generator=lambda i,q,p:circulant(i,q,p,prev=1,post=0,undirected=True)
# comm_network_generator=lambda i,q,p:fc_comm_network(i,q,p)

for N_sen in [5,10,20,40]:

# for N_sen in [40]:
    data = {'p':[],'q':[]}
    print('N_sen',N_sen)
    
    for k in range(N_trials):
        ps = np.random.rand(N_sen,2)
        q = np.array([6,6])


        # Communication network
        G = comm_network_generator(0,q,ps)

        a_list = get_adj_list(q,ps,G)

        p_hist = []
        q_hist = []

        for _ in range(N_iter):
            p_hist.append(ps.copy())
            q_hist.append(q.copy())
        
            y = vs.measurement(q,ps)

            new_ps = np.array([controller.update(ps[i],y[i],ps[nb],y[nb]) for i,nb in enumerate(a_list)])

            ps = new_ps
            
        
        data['p'].append(np.array(p_hist))
        data['q'].append(np.array(q_hist))

    data['p'] = np.array(data['p'])
    data['q'] = np.array(data['q'])
    
    with open('Moore-Distributed-{}Senor.pkl'.format(N_sen),'wb') as file:
        pkl.dump(data,file)


# for i in range(N_sen):
#     plt.scatter(p_hist[:,i,0],p_hist[:,i,1])

N_sen 5


/home/tianpeng/Desktop/fim_track/src/utils/MutualSepPathPlanning.py:58: RuntimeWarning: invalid value encountered in true_divide
  # Prevent the ps from getting too far away by doing projection


N_sen 10
N_sen 20
N_sen 40
